In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np 
import os
import pandas as pd 
# from path import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

/home/shubham/.cache/pypoetry/virtualenvs/kannada-mnist-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/shubham/.cache/pypoetry/virtualenvs/kannada-mnist-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pwd


'/home/shubham/kannada_mnist/notebooks'

In [3]:
ls

training.ipynb


In [4]:
data_path = '/home/shubham/kannada_mnist/data/'

In [5]:
train=pd.read_csv(data_path+'train.csv')
test=pd.read_csv(data_path+'Dig-MNIST.csv')
submission_set = pd.read_csv(data_path+"test.csv").iloc[:,1:]


train_data=train.drop('label',axis=1)
train_targets=train['label']

test_images=test.drop('label',axis=1)
test_labels=test['label']

# Train Test Split
train_images, val_images, train_labels, val_labels = train_test_split(train_data, 
                                                                     train_targets, 
                                                                     test_size=0.2)

# Reset Index
train_images.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)

val_images.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

test_images.reset_index(drop=True, inplace=True)
test_labels.reset_index(drop=True, inplace=True)

print("Train Set")
print(train_images.shape)
print(train_labels.shape)

print("Validation Set")
print(val_images.shape)
print(val_labels.shape)

print("Validation 2")
print(test_images.shape)
print(test_labels.shape)

print("Submission")
print(submission_set.shape)

Train Set
(48000, 784)
(48000,)
Validation Set
(12000, 784)
(12000,)
Validation 2
(10240, 784)
(10240,)
Submission
(5000, 784)


In [15]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
print("Train Distribution")
print(train_labels.value_counts(normalize = True))

print("\nSubmission Distribution")
print(test_labels.value_counts(normalize = True))

Train Distribution
0    0.100896
1    0.100646
4    0.100583
7    0.100312
6    0.100208
5    0.099937
8    0.099750
3    0.099604
2    0.099167
9    0.098896
Name: label, dtype: float64

Submission Distribution
7    0.1
6    0.1
5    0.1
4    0.1
3    0.1
2    0.1
9    0.1
1    0.1
8    0.1
0    0.1
Name: label, dtype: float64


In [23]:


# Train
train_data=torch.from_numpy(train_images.values).float().view(train_images.shape[0],28,28)
train_targets=torch.from_numpy(train_labels.values).long().view(train_labels.shape[0])

# Validation
val_data=torch.from_numpy(val_images.values).float().view(val_images.shape[0],28,28)
val_targets=torch.from_numpy(val_labels.values).long().view(val_labels.shape[0])

# Test
test_data=torch.from_numpy(test_images.values).float().view(test_images.shape[0],28,28)
test_targets=torch.from_numpy(test_labels.values).long().view(test_labels.shape[0])

submission_data=torch.from_numpy(submission_set.values).float().view(submission_set.shape[0],28,28)

print(train_targets.shape)
print(train_data.shape)

print(val_targets.shape)
print(val_data.shape)

print(test_targets.shape)
print(test_data.shape)

print(submission_data.shape)



torch.Size([48000])
torch.Size([48000, 28, 28])
torch.Size([12000])
torch.Size([12000, 28, 28])
torch.Size([10240])
torch.Size([10240, 28, 28])
torch.Size([5000, 28, 28])


In [26]:


batch_size = 8

train_set=torch.utils.data.TensorDataset(train_data, train_targets)
valid_set=torch.utils.data.TensorDataset(val_data, val_targets)
test_set=torch.utils.data.TensorDataset(test_data, test_targets)
submission_set=torch.utils.data.TensorDataset(submission_data) 

train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(valid_set, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=batch_size, 
                                          shuffle=False)

submission_loader = torch.utils.data.DataLoader(submission_set,
                                          batch_size=batch_size, 
                                          shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')



In [27]:
train_loader